# 지형 이동

소요시간: NaN

작성일시: 24.03.10 12:20:58

출처: https://school.programmers.co.kr/learn/courses/30/lessons/62050

### 접근 방법
1st
- bfs로 사다리 없이 갈 수 있는 지역을 그룹으로 나눈다.
- 각 칸의 상하좌우에 대해 다른 그룹이면 사다리의 비용을 해시에 저장한다. 이미 비용이 저장되어있으면 최솟값을 저장한다.
- ~~그룹이 n개면 사다리가 n-1개 필요하기 때문에 사다리 비용 해시를 오름차순으로 정렬하고 앞 n-1개만 더해서 반환한다.~~  
사이클이 발생하는 경우를 제외해야 한다.
- 실패

2nd
- 사이클이 발생하는 경우를 제외하고 최소 비용의 사다리 건설 -> 최소 스패닝 트리 만들기
- 각 그룹을 잇는 사다리를 입력으로 하여 크루스칼 알고리즘으로 최소 비용으로 모든 그룹을 탐색할 수 있도록 하자
- 성공

3rd
- 우선순위 큐 + BFS 사용
- 사다리 없이 갈 수 있는 칸 -> 최소 비용의 사다리를 설치하여 갈 수 있으나 아직 방문하지 않은 칸 순으로 탐색을 진행하면 그리디하게 최소 건설 비용을 찾을 수 있다.
- 성공

### 시간 복잡도 분석
행 및 열 길이 = n  
2nd 시간복잡도 = O(n^2)  
3rd 시간복잡도 = O((n^2)log(n^2))

### 새로 알게 된 것
.

### 주의할 점
.

### 기타 코멘트
.


In [ ]:
from collections import deque

def isGridIn(r, c, n):
    return 0 <= r < n and 0 <= c < n

def setGroupId(land, group, visited, height, id, i, j):
    n = len(land)
    q = deque([(i, j)])
    visited[i][j] = True
    while q:
        r, c = q.popleft()
        group[r][c] = id
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nr, nc = r + dr, c + dc
            if isGridIn(nr, nc, n) and not visited[nr][nc] and abs(land[r][c] - land[nr][nc]) <= height:
                q.append((nr, nc))
                visited[nr][nc] = True


def solution(land, height):
    n = len(land)
    group = [[0]*n for _ in range(n)]
    group_num = 0
    visited = [[False] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if not visited[i][j]:
                setGroupId(land, group, visited, height, group_num, i, j)
                group_num += 1
    ladder_hash = dict()
    for i in range(n):
        for j in range(n):
            myId = group[i][j]
            for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                nr, nc = i+dr, j+dc
                if isGridIn(nr, nc, n) and myId != group[nr][nc]:
                    neighborId = group[nr][nc]
                    key = frozenset([myId, neighborId])
                    cost = abs(land[i][j] - land[nr][nc])
                    if key in ladder_hash:
                        ladder_hash[key] = min(ladder_hash[key], cost)
                    else:
                        ladder_hash[key] = cost
    ladders = list(ladder_hash.values())
    ladders.sort()
    return sum(ladders[:group_num-1])

In [ ]:
from collections import deque

def isGridIn(r, c, n):
    return 0 <= r < n and 0 <= c < n

def setGroupId(land, group, visited, height, id, i, j):
    n = len(land)
    q = deque([(i, j)])
    visited[i][j] = True
    while q:
        r, c = q.popleft()
        group[r][c] = id
        for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            nr, nc = r + dr, c + dc
            if isGridIn(nr, nc, n) and not visited[nr][nc] and abs(land[r][c] - land[nr][nc]) <= height:
                q.append((nr, nc))
                visited[nr][nc] = True

def find_parent(parent, x):
    if x != parent[x]:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]

def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b

def solution(land, height):
    n = len(land)
    group = [[0]*n for _ in range(n)]
    group_num = 0
    visited = [[False] * n for _ in range(n)]
    for i in range(n):
        for j in range(n):
            if not visited[i][j]:
                setGroupId(land, group, visited, height, group_num, i, j)
                group_num += 1
    ladder_hash = dict()
    for i in range(n):
        for j in range(n):
            myId = group[i][j]
            for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                nr, nc = i+dr, j+dc
                if isGridIn(nr, nc, n) and myId != group[nr][nc]:
                    neighborId = group[nr][nc]
                    key = frozenset([myId, neighborId])
                    cost = abs(land[i][j] - land[nr][nc])
                    if key in ladder_hash:
                        ladder_hash[key] = min(ladder_hash[key], cost)
                    else:
                        ladder_hash[key] = cost

    parent = list(range(group_num))
    answer = 0
    ladder_num = 0
    for edges in sorted(ladder_hash, key=lambda x: ladder_hash[x]):
        a, b = edges
        cost = ladder_hash[edges]
        if find_parent(parent, a) != find_parent(parent, b):
            union_parent(parent, a, b)
            answer += cost
            ladder_num += 1
            if ladder_num == group_num - 1:
                break
    return answer

In [ ]:
import heapq

def isGridIn(r, c, n):
    return 0 <= r < n and 0 <= c < n

def solution(land, height):
    n = len(land)
    RDLU = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    visited = [[False] * n for _ in range(n)]
    q = []
    heapq.heappush(q, (0, 0, 0))
    answer = 0
    while q:
        cost, r, c = heapq.heappop(q)
        curHeight = land[r][c]
        if not visited[r][c]:
            visited[r][c] = True
            answer += cost
            for dr, dc in RDLU:
                nr, nc = r + dr, c + dc
                if isGridIn(nr, nc, n):
                    nextHeight = land[nr][nc]
                    nextCost = abs(nextHeight - curHeight) if abs(nextHeight - curHeight) > height else 0
                    heapq.heappush(q, (nextCost, nr, nc))
    return answer